In [12]:
from pathlib import Path
import csv
import sys
from feat import Detector
import logging

In [2]:
path_to_save_local = '/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/proj_fex_software_comparison/outputs/'
path_to_save_server = '/Volumes/LYA/LYA_Lab/FEX_Substudy1/BIDS_dataset/code/FEX_AU_comparison_Marie/FEX_AU_outputs/'

In [3]:
file_all_folders_server = '/Volumes/LYA/LYA_Lab/FEX_Substudy1/BIDS_dataset/code/FEX_AU_comparison_Marie/FEX_AU_code/all_folders.txt'
file_all_folders_local = '/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/proj_fex_software_comparison/mydata/test_data/all_folders_local.txt'

In [22]:
def log(msg): # helper function
    logging.info(msg)  
    print(msg)

def run_pyfeat (which_algorithm, path_all_folders_file, path_to_save):
    '''
    Function to process all video data with pyfeat.
    Assumed input data from multiple folders, one folder per participant, doesn't matter how many videos in each folder.
    Saves csv outputs: pyfeat output file for each video; "master" file with info on how many frames failed in each video.
    Parameters:
    - which_algorithm ('xgb' or 'svm'): specify which algorithm, available in pyfeat to use for prediction.
    - path_all_folders_file: full path to a .txt file that lists full paths to participant folder (one path per line).
    (generate it with find path/to/data -mindepth 1 -maxdepth 1 -type d | sort > all_folders.txt)
    - path_to_save: path to root folder to save output data.
    '''
    assert which_algorithm in ['xgb', 'svm']

    log_path = f"{path_to_save}/outputs_pyfeat_{which_algorithm}/mylog_pyfeat_{which_algorithm}_jupyter.log"

    # vibecoding the shit out of the logging thing
    
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s | %(levelname)s | %(message)s",
        handlers=[
            #logging.StreamHandler(sys.stdout),  # console
            logging.FileHandler(log_path, mode="w", encoding="utf-8"),  # file
        ]
    )

    try:
        log(f'\n\n==============================\n START PYFEAT ({which_algorithm.upper()}) PROCESSING \n==============================\n')   
        log('\nPrep work...')
            
        if which_algorithm == 'xgb':
            # define paths
            my_path_to_save = path_to_save + 'outputs_pyfeat_xgb'
            # load pyfeat detector module
            detector = Detector(au_model='xgb')
            
        elif which_algorithm == 'svm':
            # define paths
            my_path_to_save = path_to_save + 'outputs_pyfeat_svm'
            # load pyfeat detector module
            detector = Detector(au_model='svm')    
    
        # prep file to save frames data
        frames_file = f"{my_path_to_save}/frames_pyfeat_{which_algorithm}.csv"
        with open(frames_file, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["name", "total", "processed_frames",])
        
        log('\nPrep work DONE\n')

        lines = list(open(path_all_folders_file, "r"))
        total_lines = len(lines)

        # read all folders data, iterate
        with open(path_all_folders_file, "r") as f:            
            for ii, line in enumerate(f, start=1):
                print(line)
                    
                # work with data folder by folder:
                
                folderpath = Path(line.strip())
                name_base = (Path(folderpath).name)
    
                files_in_dir = [
                    p for p in Path(folderpath).iterdir()
                    if p.is_file() and p.suffix.lower() == ".mp4"
                ]
                
                howmanyvidsindir = len(files_in_dir)
    
                log("\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
                log(f"+++++++++++++++++++++++++++++++++++++++ Processing FOLDER {ii} out of {total_lines}: {name_base} ++++++++++++++++++++++++++++")
                log("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")
                    
                # iterate through all .mp4 files in folder
                for i, path_to_file in enumerate(files_in_dir, start=1): # so that counting starts at 1 and not 0
                    if path_to_file.suffix.lower() == ".mp4": 
    
                        filename_stem = path_to_file.stem
                        filename = path_to_file.name
                        
                        log(f"---------------------------------------------------------------- \n Processing file {i} out of {howmanyvidsindir}: {filename} \n in folder {ii} out of {total_lines}: \n ---------------------------------------------------------------")
    
                        # for this file:
                        
                        # 1. Do pyfeat processing
                        
                        log(f"\n Doing {which_algorithm.upper()} processing...")
                        video_prediction = detector.detect_video(str(path_to_file), data_type="video")
                        log(f"\n {which_algorithm.upper()} processing: DONE")                    
        
                        # 2. Save pyfeat output
                        
                        log('\n Saving output files...')
    
                        path_to_save_vid = f"{my_path_to_save}/{name_base}/output_pyfeat_{which_algorithm}_{filename_stem}.csv"            
                        video_prediction.to_csv(path_to_save_vid, index=True)
                        
                        log('\n Saving output files: DONE') 
    
                        # 3. Save frames stats
    
                        log('\n Saving frames stats...')
                
                        total_frames_vid = video_prediction.shape[0]
                        processed_frames_vid = video_prediction.FaceScore.isna().value_counts()[False]
                
                        with open(frames_file, "a", newline="") as f:
                            writer = csv.writer(f)
                            writer.writerow([
                                filename,
                                total_frames_vid,
                                processed_frames_vid
                            ])
                            
                        log('\n Saving frames stats: DONE')
    
                        log(f"\n DONE file: {filename}\n")
    
                log(f"\n+++++++++++++++++++++++++++++++++++++\n DONE FOLDER: {name_base} \n+++++++++++++++++++++++++++++++++++++\n")
                        
        log(f'\n\n==============================\n DONE PYFEAT ({which_algorithm.upper()}) PROCESSING \n=============================\n')
    except KeyboardInterrupt:
        log("Script interrupted by user!")

    except Exception as e:
        logging.exception(f"Script crashed: {e}")  # logs exception with traceback
    
    finally:
        logging.shutdown()  # flush everything immediately
    
    return(0)


In [23]:
run_pyfeat(which_algorithm='xgb', path_all_folders_file=file_all_folders_local, path_to_save=path_to_save_local)



 START PYFEAT (XGB) PROCESSING 


Prep work...

Prep work DONE

/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/proj_fex_software_comparison/mydata/test_data/sub-9473


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++ Processing FOLDER 1 out of 2: sub-9473 ++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

---------------------------------------------------------------- 
 Processing file 1 out of 5: 9473_GOPR1255_heat2_trial1.mp4 
 in folder 1 out of 2: 
 ---------------------------------------------------------------

 Doing XGB processing...


  6%|████████████▉                                                                                                                                                                                                  | 15/241 [00:13<03:22,  1.12it/s]


Script interrupted by user!


0